In [93]:
# Advent Puzzle Day 3
import numpy as np
import os

In [94]:
os.chdir('/Users/justinehly/GitHub/advent_of_code/day 4/')
os.getcwd()

'/Users/justinehly/GitHub/advent_of_code/day 4'

In [95]:
input = 'day_4_input.txt'
with open(input) as f:
    lines = f.readlines()

In [96]:
DRAW = [lines[0][:-1]]
DRAW = DRAW[0].split(',')

In [97]:

boards = lines[1:]
boards = [i[:-1] for i in boards if i != '\n']
squid = []
for i,x in enumerate(boards):
    squid.append(x.split())
BINGO = []
b = 0
for i in range(0, len(squid), 5):
    BINGO.append(squid[i:i+5])
BINGO = np.reshape(BINGO, (5,5,100))


In [100]:
BINGO2 = np.zeros((5,5,100))

In [111]:
marker = np.where(BINGO == '57')
BINGO2[marker] = 1
BINGO2

array([[[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]])